In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [2]:
os.environ['TOKEN'] = "AAAAAAAAAAAAAAAAAAAAANcAaQEAAAAA84hzZvsLo8rITqxhQJoQHVAd5wI%3DfD5dXkdwihlc4K08dNi3y84D4lVhjiOIREITen9vVe2UVYxFh7"

In [3]:
def auth():
    return os.getenv('TOKEN')

In [4]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [26]:
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [27]:

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()



In [28]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "#vaccine lang:en -is:retweet place_country:US has:geo"
start_time = "2021-03-01T00:00:00.000Z"
end_time = "2021-03-31T00:00:00.000Z"
max_results = 10

In [29]:
url = create_url(keyword, start_time,end_time, max_results)

In [30]:
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 200


In [31]:
print(json.dumps(json_response, indent=4, sort_keys=True))

{
    "data": [
        {
            "author_id": "756155529794297856",
            "conversation_id": "1377045257372758021",
            "created_at": "2021-03-30T23:49:08.000Z",
            "geo": {
                "place_id": "4a9aecc7ad6b85e6"
            },
            "id": "1377045257372758021",
            "lang": "en",
            "public_metrics": {
                "like_count": 1,
                "quote_count": 0,
                "reply_count": 0,
                "retweet_count": 0
            },
            "reply_settings": "everyone",
            "source": "Twitter for iPhone",
            "text": "Have you signed up to get your #COVID19 #vaccine for when eligibility opens up April 5th? https://t.co/h2jsJ8fzpU"
        },
        {
            "author_id": "963546977706364933",
            "conversation_id": "1377044106665418757",
            "created_at": "2021-03-30T23:44:33.000Z",
            "geo": {
                "place_id": "102251caf154e000"
            },
     

In [32]:
json_response['data'][0]['created_at']

'2021-03-30T23:49:08.000Z'

In [12]:
json_response['meta']['result_count']

10

In [13]:
with open('data.json', 'w') as f:
    json.dump(json_response, f)

In [41]:
# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['conversation_id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

In [15]:
json_response['data']

[{'lang': 'en',
  'author_id': '756155529794297856',
  'source': 'Twitter for iPhone',
  'id': '1377045257372758021',
  'reply_settings': 'everyone',
  'public_metrics': {'retweet_count': 0,
   'reply_count': 0,
   'like_count': 1,
   'quote_count': 0},
  'conversation_id': '1377045257372758021',
  'created_at': '2021-03-30T23:49:08.000Z',
  'geo': {'place_id': '4a9aecc7ad6b85e6'},
  'text': 'Have you signed up to get your #COVID19 #vaccine for when eligibility opens up April 5th? https://t.co/h2jsJ8fzpU'},
 {'lang': 'en',
  'author_id': '963546977706364933',
  'source': 'Twitter for iPhone',
  'id': '1377044106665418757',
  'reply_settings': 'everyone',
  'public_metrics': {'retweet_count': 1,
   'reply_count': 2,
   'like_count': 7,
   'quote_count': 1},
  'conversation_id': '1377044106665418757',
  'created_at': '2021-03-30T23:44:33.000Z',
  'geo': {'place_id': '102251caf154e000'},
  'text': 'Second Covid vaccine! #Covid_19 #vaccine https://t.co/gFjt5EDpPY'},
 {'lang': 'en',
  'auth

In [42]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        auther_id = tweet['auther_id']
        #name = tweet['name']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [auther_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

In [40]:
append_to_csv(json_response, "data.csv")

# of Tweets added from this response:  10


In [21]:
url

('https://api.twitter.com/2/tweets/search/all',
 {'query': '#vaccine lang:en -is:retweet place_country:US has:geo',
  'start_time': '2021-01-01T00:00:00.000Z',
  'end_time': '2021-01-01T23:59:59.000Z',
  'max_results': 10,
  'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
  'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
  'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
  'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
  'next_token': None})

In [43]:
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "#vaccine lang:en -is:retweet place_country:US has:geo"
start_list =    ['2021-01-01T00:00:00.000Z',
                 '2021-01-02T00:00:00.000Z',
                 '2021-01-03T00:00:00.000Z'
                ]

end_list =      [
                 '2021-01-01T23:59:59.000Z',
                 '2021-01-02T23:59:59.000Z',
                 '2021-01-03T23:59:59.000Z'
                 ]
max_results = 10

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['auther_id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 10 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(0)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(0)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foshtsfjzeh00sp7ftpaxipamqnp8d
Start Date:  2021-01-01T00:00:00.000Z


KeyError: 'auther_id'